In [1]:
'Based on https://github.com/amir-abdi/keras_to_tensorflow'

from keras.models import load_model
import tensorflow as tf
from keras import backend as K

Using TensorFlow backend.


In [2]:
input_h5_name = 'mnist_cnn.h5'
output_pb_name = 'mnist_cnn.pb'
num_output = 1
prefix_output_node_names_of_final_network = 'output_node'

In [14]:
# Load keras model and rename output
K.set_learning_phase(0)
net_model = load_model(input_h5_name)

pred = [None]*num_output
pred_node_names = [None]*num_output
for i in range(num_output):
    pred_node_names[i] = prefix_output_node_names_of_final_network+str(i)
    pred[i] = tf.identity(net_model.output[i], name=pred_node_names[i])
print('output nodes names are: ', pred_node_names)

('output nodes names are: ', ['output_node0'])


In [16]:
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io

sess = K.get_session()
constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), pred_node_names)
graph_io.write_graph(constant_graph, '.', output_pb_name, as_text=False)

INFO:tensorflow:Froze 8 variables.


INFO:tensorflow:Froze 8 variables.


Converted 8 variables to const ops.


'./mnist_cnn.pb'

In [17]:
'Tensorboard Visualization'
'Sorry can not remember where I got the source from :('

import tensorflow as tf
import os
import numpy as np
from tensorflow.python.platform import gfile
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [18]:
with tf.Session() as persisted_sess:
  print("load graph")
#   with gfile.FastGFile(output_pb_name,'rb') as f:
with gfile.FastGFile(output_pb_name,'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    persisted_sess.graph.as_default()
    tf.import_graph_def(graph_def, name='')
#     show_graph(persisted_sess.graph)
    show_graph(tf.get_default_graph().as_graph_def())

load graph
